In [1]:
import pandas as pd

# Load the canonical results data released by OP

https://docs.google.com/spreadsheets/d/1h6hPbRE5w8iJxVhxOSYvkMYz-HC6-lePOegXw-usW5Q/edit#gid=583271201

In [2]:
df_results = pd.read_csv("csv/RetroPGF3 Results - Canonical.csv", thousands=",")
df_results.index = df_results['Project Profile'].str.replace('https://vote.optimism.io/retropgf/3/application/','')
df_results.index.name = 'Project ID'
df_results.rename(columns={
    'ballots': 'Result: # Ballots',
    'Quorum reached': 'Result: Quorum Reached',
    'median amount': 'Result: Median OP',
    'OP Received': 'Result: Received OP'
}, inplace=True)
df_results.drop(columns=['Project Name', 'Project Profile'], inplace=True)
df_results.iloc[0]

Result: # Ballots                95
Result: Median OP          668000.0
Result: Quorum Reached         True
Result: Received OP       663853.62
Name: 0x2a1d784639add63bb313c3836b5dc1421d0a775d4cf7459c97bfbee95f39e26f, dtype: object

# Load the growthepie data to capture other funding sources

https://www.growthepie.xyz/optimism-retropgf-3

In [3]:
df_gtp = pd.read_csv(
    "csv/RetroPGF3 Results - growthepie.csv", 
    usecols=['project_id', 'vc_funding', 'has_token'], 
    index_col=0
)
df_gtp['GTP: Has VC Funding'] = (df_gtp['vc_funding'].isna() != True) * 1
df_gtp.rename(columns={
    'vc_funding': 'GTP: VC Funding Amount',
    'has_token': 'GTP: Has Token'
}, inplace=True)
df_gtp.iloc[0]

GTP: VC Funding Amount    NaN
GTP: Has Token            NaN
GTP: Has VC Funding         0
Name: 0xef65b1e277e1c7398bbd8a10c623d53dbbb64474ea68b6484e9ccf5c9c0cc494, dtype: object

# Load the OSO snapshot at the start of voting

https://docs.google.com/spreadsheets/d/1uOmzUHdbGu_OGnXTYrLNmKQMJx2UZObJm-KRBmPnbCc/edit?usp=sharing

In [7]:
df_projects = pd.read_csv("csv/RetroPGF3 Projects - OSO Snapshot.csv", index_col=0)
df_projects['OSO: Has Profile'] = (df_projects['OSO Slug'].isna() != True) * 1
df_projects.drop(columns=['OSO Slug'], inplace=True)
df_projects.rename(
    columns={
        c: f"Meta: {c}" for c in df_projects.columns[:5]
    },
    inplace=True
)
df_projects.rename(
    columns={
        c: f"OSO: {c}" for c in df_projects.columns if ":" not in c
    },
    inplace=True
)
df_projects.iloc[0]

Meta: Project Name                                                                     0xDeFiTH
Meta: Applicant Type                                                                 INDIVIDUAL
Meta: Website                                                      https://twitter.com/0xDeFiTH
Meta: Bio                                     I'm doing the ETH and DeFi educational content...
Meta: Payout Address                                 0xA84fa9e80E2b01C48fBc4dfdc8b5aC57F12a7e2B
Category: Collective Governance                                                               0
Category: Developer Ecosystem                                                                 0
Category: End User Experience and Adoption                                                    1
Category: OP Stack                                                                            0
Funding: Governance Fund                                                                      0
Funding: Other                          

# Join the three datasets and dump them

In [8]:
df = df_results.join(df_projects.join(df_gtp))
df.sort_values(by=['Result: Received OP', 'Result: # Ballots'], inplace=True, ascending=False)
df.iloc[0]

Result: # Ballots                                                                            95
Result: Median OP                                                                      668000.0
Result: Quorum Reached                                                                     True
Result: Received OP                                                                   663853.62
Meta: Project Name                                                               Protocol Guild
Meta: Applicant Type                                                                    PROJECT
Meta: Website                                             https://protocol-guild.readthedocs.io
Meta: Bio                                     A collective of Ethereum's active core protoco...
Meta: Payout Address                                 0x728D29E9E06cE5d846242692dF05467076c19849
Category: Collective Governance                                                               0
Category: Developer Ecosystem           

In [9]:
df.to_csv("csv/RetroPGF3 Results - ConsolidatedData.csv")